# CE888 Assignment 2 for Demo

This code is written to demonstarte a classifier using an autoencoder for ce888 assignment 2.

**Author**          : Tomoko Ayakawa<br> 
**Created on**      : 29 March 2019<br> 
**Last modified on**: 20 April 2019<br> 

In [33]:
# import open source libraries
import numpy as np
import sys, keras

# import original libraries
sys.path.append("../")
sys.path.append("./sub")

from conf import myVariables as VAR
import load_data as DATA
import preprocess as PREP
import autoencoder as AE
import mlp as MLP
import grid_search as GS
import comparison as CMP

# Automatically reload changed modules before executing each line
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Prepare the data
### 1-1. Select the data to load

In [35]:
data_list={0: "human activity", 1: "spam", 2: "phishing"}
data_id=int(input(data_list))

col_names, features_df, targets_df, data_df, timestamp=DATA.load_data(data_id=data_id)
unique_labels=DATA.verify_data(data_df, targets_df, dispaly_range=False)

{0: 'human activity', 1: 'spam', 2: 'phishing'}0
Number of NaN: 0
Data shape:  (7352, 562)
Target labels: ['1' '2' '3' '4' '5' '6']
Class distribution: ['0.17', '0.15', '0.13', '0.17', '0.19', '0.19']


### 1-2. Obtain small data

In [36]:
features, classes=PREP.get_small_data(features_df, targets_df)
pic_file=timestamp+"(%d)" % len(features)

Enter the number of samples to use (Total: 7352): 
 - Number of features: 561
 - Minimul data size : 4538
 - Specified size    : 4538



### 1-3. Pre-process the data
Fit the scaler and transform the data.<br>
When the scaler ID is not specified, MinMaxScaler will be applied.

In [37]:
scl, features_nrm=PREP.pre_processing(features, display_result=False)

Select the scaler 0 (None), 1 (MinMax), 2 (Quantile), 3 (Standard): 
Scaler: MinMaxScaler(copy=True, feature_range=(0, 1))


### 1-4. Split the data

In [38]:
X, y=features_nrm, classes
X_tr, X_te, y_tr, y_te=PREP.split_data(features_nrm, classes)

Enter the test_size (0<=test_size<1): 
Training: 3630, Test: 908



## 4. Build an Autoencoder
### 4-1. Define the parameters

In [ ]:
ae_layers, mode, act, opt, loss, dropout, \
    epochs, verbose, summary_display=AE.get_parameters(data_id)
pic_file=timestamp+"(%d_%.1f)" % (len(features), dropout)

Define 9 parameters for the autoencoder.
When skipped (push enter) or an invalid value is given, the default value will be used.
[Parameter 1/9: Autoencoder type] 0:Basic or 1:Stacked (default=1): 
[Parameter 2/9: Number of neurons in each layer] Integers separated by comma (default=500,400,300,200,100,50,25,10): 
[Parameter 3/9: Activation function] 0:relu 1:sigmoid 2:tanh 3:softmax (default=relu): 
[Parameter 4/9: Optimiser] 0:adam 1:sdg (default=adam): 
 - Learning rate (default=0.001000): 
[Parameter 5/9: Loss function] 0:mse 1:mean_absolute_error 2:mean_squared_logarithmic_error 3:categorical_crossentropy (default=mse): 
[Parameter 6/9: Dropout rate] 0<=rate<1 (default=0.000000): 
[Parameter 7/9: Training epochs] (default=100): 
[Parameter 8/9: Verbose] 0:False or 1:True (default=0): 
[Parameter 9/9: Summary display] 0:False or 1:True (default=0): 

Parameters for the autoencoder are
 1. Mode: 1(Stacked)
 2. Layers: [500, 400, 300, 200, 100, 50, 25, 10]
 3. Activation function: re

### 4-2. Train the autoencoder

In [ ]:
encoder, histories=AE.autoencoder(X, layers=ae_layers, mode=mode, act=act, opt=opt, 
                   loss=loss, dropout=dropout, epochs=epochs, verbose=verbose, 
                   summary_display=summary_display)

Training Layer 1/8 ...


### 4-3. Display the training loss history

In [ ]:
AE.plot_ae_loss_history(histories, mode, pic_file)

### 4-4. Extract features from the datasets

In [ ]:
X_all_cmp=encoder.predict(X)
X_tr_cmp=encoder.predict(X_tr)
X_te_cmp=encoder.predict(X_te)

print("The number of compressed features:", len(X_all_cmp[0]))

## 5. Build a discriminative neural network

### 5-1. Grid Search for optimal parameter
#### (1) Define the parameters
|parameter               |default    |description                                              |
|------------------------|-----------|---------------------------------------------------------|
|hidden_layer_sizes      |(100,)     |i-th element: numb of neurons in the i-th hidden layer   |
|activation              |"relu"     |activation function                                      |
|solver                  |"adam"     |weight optimasation                                      |
|learning_rate_init      |0.001      |step-size in weights update (solver=’sgd’ or ‘adam’)     |
|max_iter                |200        |max num of iterations                                    |

In [ ]:
#act, h_num, max_itr, lr, solver, splits=GS.get_parameters()

#### (2) Grid Search

In [ ]:
#param_grid, clf=GS.parameter_grid(activation=act, \
#                    hidden_layer_sizes=(h_num,), max_iter=max_itr, \
#                    learning_rate_init=lr, solver=solver)
#if param_grid!=1: GS.grid_search(X_all_cmp, y, clf, param_grid, grid_splits=splits)

### 5-2. Define the parameters for MLP

In [ ]:
finetune, h_num, h_act, out_act, opt, loss, epochs, val_rate, \
    verbose, summary_display=MLP.get_parameters()

### 5-3. Train the MLP

In [ ]:
model=MLP.build_mlp(encoder, num_in=ae_layers[-1], num_out=len(unique_labels), \
            finetune=finetune, h_num=h_num, h_act=h_act, \
            out_act=out_act, opt=opt, loss=loss, \
            summary_display=summary_display)
histories=MLP.train_mlp(X, y, model, epochs=epochs, \
            val_rate=val_rate, verbose=verbose)

### 5-4. Display the training loss history

In [ ]:
MLP.plot_mlp_loss_history(histories, pic_file)

## 6. Evaluation
### 6-1. Cross Validation

In [ ]:
accs, fscores=MLP.cross_validation(model, X, y, unique_labels, epochs=epochs)

### 6-2. Train, predict and evaluate the classifier

In [ ]:
pred=MLP.evaluation(model, X_tr, y_tr, X_te, y_te, unique_labels, pic_file)

## 7. Comparison with other thechniques
### 7-1. Performance comparison with DT, NB, SVM, NN

In [ ]:
%matplotlib inline
# original data (uncompressed)
CMP.performance_comparison(pred, X_tr, X_te, y_tr, y_te, \
                           timestamp+"(%d)" % len(features), data_id)

### 7-2. Plot 3 dimensions obtained by PCA and Autoencoder

In [ ]:
# if the compressed features have more than three dimensions, retrain the autoencoder
if ae_layers[-1]!=3:
    if input("Retrain the autoencoder? (y/n): ")=="y":
        ae_layers.append(5)
        ae_layers.append(3)
        re_encoder, histories=AE.autoencoder(X, layers=ae_layers, mode=mode, act=act, 
                           opt=opt, loss=loss, dropout=dropout, epochs=epochs, 
                           verbose=verbose, summary_display=summary_display)
        X_all_cmp=re_encoder.predict(X)

        print("The number of compressed features:", len(X_all_cmp[0]))

In [ ]:
# the number of unirue values in each dimension
for x, t in zip([X, X_all_cmp], ["PCA    ", "Encoder"]):
    print(t, ":", len(x), 
          "\t| axis-1:", len(np.unique(x[:,0])), 
          "\t| axis-2:", len(np.unique(x[:,1])),
          "\t| axis-3:", len(np.unique(x[:,2])))

In [ ]:
%matplotlib notebook
if ae_layers[-1]==3: CMP.plot_3D(X, X_all_cmp, y)

# ===== End of the code =====